In [1]:
from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import skforecast
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cargo Dataset principal

In [2]:
df = pd.read_csv('../../data/Top100_prediction_con.csv')

In [3]:
# Lista para guardar resultados
resultados = []

# Copia del DataFrame original
df_original = df.copy()

# Seleccionar los primeros 5 productos únicos
productos = list(df_original['item'].unique())

for producto in productos:
    # Filtrar el DataFrame para el producto actual
    df_filtrado = df_original[(df_original['store_code'] == 'NYC_1') & (df_original['item'] == producto)]
    
    # Verificar si hay suficientes datos
    if len(df_filtrado) < 30 + 14:  # Necesitas al menos 44 registros
        print(f"Advertencia: No hay suficientes datos para el producto {producto}. Saltando...")
        continue
    
    # Preparar los datos
    ts_predict = df_filtrado[['date', 'sales', 'weekday', 'event']]
    ts_predict = pd.get_dummies(data=ts_predict, columns=['weekday'], dtype=int)
    ts_predict['date'] = pd.to_datetime(ts_predict['date'])
    ts_predict.sort_values('date', ascending=True, inplace=True)
    ts_predict.set_index('date', inplace=True)
    ts_predict = ts_predict.asfreq('D')

    # Separar en entrenamiento y prueba
    y, exog = ts_predict['sales'], ts_predict.drop(columns=['sales'])
    y_train, y_test = y[:-30], y[-30:]
    exog_train, exog_test = exog[:-30], exog[-30:]

    # Inicializar el modelo Forecaster
    forecaster = ForecasterAutoreg(
        regressor=RandomForestRegressor(max_depth=None,
                                        min_samples_leaf=2,
                                        min_samples_split=2,
                                        n_estimators=50),
        lags=7
    )
    
    # Ajustar el modelo
    forecaster.fit(y=y_train, exog=exog_train)

    start_date = '2016-04-25' 
    end_date ='2016-05-30'
    # Crear un rango de fechas
    date_range = pd.date_range(start=start_date, end=end_date)
    # Crear el dataframe
    df1 = pd.DataFrame(date_range, columns=['date'])
    df1['date'] = pd.to_datetime(df1['date'])
    df1['weekday'] = df1['date'].dt.day_name()
    df1['event'] = 0
    df1 = pd.get_dummies(data=df1, columns=['weekday'], dtype=int)
    df1.sort_values('date', inplace=True, ascending=True)
    df1.set_index('date', inplace=True)
    df1 = df1.asfreq('D')
    exog_test = pd.concat([exog_test, df1], ignore_index=False)
    
    
    # Predecir
    predicciones = forecaster.predict(steps=60, exog=exog_test)
    
    # Almacenar resultados
    resultados.append({
        'producto': producto,
        'prediccion': predicciones.values,
        'test': y_test.values
    })
    
    #print(f"Predicciones generadas para el producto: {producto}")


/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/home/haroldavis/Projects/DS-MARKET-TFM/env/lib/python3.10/site-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider 

In [4]:
tiendas_inventario = pd.DataFrame(resultados)
tiendas_inventario['test_suma'] = tiendas_inventario['test'].apply(sum)
suma = tiendas_inventario['test_suma'].sum()
tiendas_inventario['prediccion'] = tiendas_inventario['prediccion'].apply(lambda x: np.round(x, decimals=0))
tiendas_inventario['tiendas_inventario_test'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[:30]))
tiendas_inventario['tiendas_inventario_prediction'] = tiendas_inventario['prediccion'].apply(lambda x: sum(x[30:]))
tiendas_inventario[['producto','test_suma','tiendas_inventario_test','tiendas_inventario_prediction']]

,producto,test_suma,tiendas_inventario_test,tiendas_inventario_prediction
0,ACCESORIES_1_108,28,40.0,39.0
1,HOME_&_GARDEN_1_027,54,88.0,94.0
2,HOME_&_GARDEN_1_053,120,92.0,87.0
3,HOME_&_GARDEN_1_140,44,51.0,48.0
4,HOME_&_GARDEN_1_177,31,87.0,81.0
...,...,...,...,...
95,SUPERMARKET_3_499,411,401.0,427.0
96,ACCESORIES_1_158,124,140.0,142.0
97,SUPERMARKET_3_282,676,908.0,896.0
98,ACCESORIES_1_354,252,198.0,191.0


In [5]:
total_inventario_productos_real = tiendas_inventario['test_suma'].sum()
total_inventario_productos_predicho = tiendas_inventario['tiendas_inventario_test'].sum()
print(total_inventario_productos_real)
print(total_inventario_productos_predicho)
print(total_inventario_productos_real - total_inventario_productos_predicho)

24477
28616.0
-4139.0


In [6]:
file_name = "NYC_1_unidades.xlsx"
tienda.to_excel(file_name)

In [4]:
pd.DataFrame(resultados)

,producto,prediccion,test
0,ACCESORIES_1_108,"[1.9256190476190478, 1.5232857142857141, 1.315...","[1, 3, 0, 0, 1, 1, 1, 0, 1, 2, 4, 0, 0, 0, 1, ..."
1,HOME_&_GARDEN_1_027,"[3.1266666666666674, 2.5348571428571427, 2.743...","[0, 2, 2, 3, 0, 2, 0, 4, 0, 1, 2, 1, 2, 4, 1, ..."
2,HOME_&_GARDEN_1_053,"[3.5529761904761896, 3.596, 2.616333333333334,...","[6, 4, 7, 5, 2, 1, 6, 6, 7, 3, 2, 3, 2, 1, 5, ..."
3,HOME_&_GARDEN_1_140,"[1.728333333333333, 2.344333333333333, 1.57104...","[2, 0, 3, 1, 1, 1, 0, 3, 2, 2, 1, 1, 0, 3, 3, ..."
4,HOME_&_GARDEN_1_177,"[3.795095238095237, 4.072190476190476, 2.30442...","[3, 1, 1, 1, 2, 5, 2, 3, 6, 6, 0, 1, 0, 0, 0, ..."
...,...,...,...
95,SUPERMARKET_3_499,"[14.994380952380952, 7.54552380952381, 12.9831...","[0, 0, 18, 27, 15, 18, 33, 18, 13, 33, 7, 8, 1..."
96,ACCESORIES_1_158,"[7.0575714285714275, 3.9037380952380953, 4.868...","[4, 11, 4, 8, 5, 3, 6, 2, 7, 2, 2, 1, 3, 4, 4,..."
97,SUPERMARKET_3_282,"[29.12, 36.50914285714286, 20.60634126984127, ...","[27, 17, 15, 22, 19, 17, 21, 43, 42, 30, 27, 2..."
98,ACCESORIES_1_354,"[9.688253968253969, 8.710738095238096, 5.85266...","[13, 8, 8, 6, 8, 5, 6, 11, 11, 9, 1, 9, 7, 8, ..."


## Predigo 30 dias(los de test) 